<a href="https://colab.research.google.com/github/Huangjian2013/ai-demo/blob/main/rag/10-LangChain-pdf-rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain --quiet
!pip install langchain-openai --quiet
!pip install langchain-community --quiet
!pip install pypdf --quiet
!pip install chromadb --quiet
!pip install tiktoken --quiet

In [3]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA

In [4]:
model = ChatOpenAI(api_key = userdata.get('REAL_OPENAI_KEY'))

In [13]:
loader = PyPDFLoader("sample_data/极越01.pdf")

documents = loader.load();

In [14]:

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [15]:
embeddings = OpenAIEmbeddings(api_key = userdata.get('REAL_OPENAI_KEY'))
docsearch = Chroma.from_documents(docs, embeddings)

In [ ]:

retriever=docsearch.as_retriever()

question = "极越01车型尺寸"

retrieved_documents = retriever.get_relevant_documents(question)

for document in retrieved_documents:
    print("-------------------------------------")
    print(document.page_content)


In [ ]:

qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever = retriever)

qa.invoke(question)